# VGG16 Image Processing

In this notebook I use VGG16 as a feature-extractor for pictures of parcels taken in detroit. I put a simple fully-connected top on VGG16 which outputs the probability that the picture shows blight (0 = blighted, 1 = not_blighted). 

In [29]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model, Sequential
import keras.backend as K
from keras.optimizers import SGD

First, instantiate the VGG16 model.

In [43]:
model = VGG16(weights='imagenet', include_top=False)

The images are stored in a directory image_data. This has subdirectories train and test, each of which have subdirectories blighted and not_blighted. I'll use a Keras ImageGenerator class to pull images. For training, I'll allow zoomed images and flipped images so the net will rarely see the same image twice. For testing, I'll just show the original images.

In [44]:
train_datagen = image.ImageDataGenerator()

In [45]:
test_datagen = image.ImageDataGenerator()

In [46]:
train_gen = train_datagen.flow_from_directory('image_data/train',
                                              target_size=(224,224),
                                              batch_size=128,
                                              class_mode='binary')

Found 9301 images belonging to 2 classes.


In [47]:
test_gen = test_datagen.flow_from_directory('image_data/test',
                                            target_size=(224,224),
                                            batch_size=128,
                                            class_mode='binary')

Found 2326 images belonging to 2 classes.


Next, I'll specify the top model interface and hook it up to my vgg16 instance.

In [48]:
top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.8))
top_model.add(Dense(1, activation='sigmoid'))

In [49]:
inputs = Input(shape=(224,224,3))
x = model(inputs)
preds = top_model(x)
combined_model = Model(input=inputs, output=preds)

In [50]:
combined_model.layers[1].layers

Note the we want to keep the layers of vgg16 frozen and only train the weights on the top model.

In [51]:
for l in combined_model.layers[1].layers:
    l.trainable = False

I want to use precision, recall, fscore, and accuracy as my metrics. Keras currently implements accuracy but not the others so I'll define custom metrics and use these in compiling the combined_model.

In [9]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [13]:
def fscore(y_true, y_pred, beta=1):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [52]:
combined_model.compile(loss='mse',
                       optimizer='sgd',
                       metrics=['accuracy', precision, recall, fscore])

Finally, train the combined model for 16 epochs, evaluating on the test data as we go along. This takes about 4 hours, about 15 minutes per epoch.

In [53]:
combined_model.fit_generator(train_gen,
                             samples_per_epoch=2048,
                             nb_epoch=16,
                             validation_data=test_gen,
                             nb_val_samples=512,
                             verbose=2)

Epoch 1/16
908s - loss: 0.5142 - acc: 0.4790 - precision: 0.5263 - recall: 0.1291 - fscore: 0.1751 - val_loss: 0.4963 - val_acc: 0.5039 - val_precision: 0.5000 - val_recall: 0.0076 - val_fscore: 0.0149
Epoch 2/16
897s - loss: 0.4490 - acc: 0.5474 - precision: 0.6945 - recall: 0.1499 - fscore: 0.2409 - val_loss: 0.4815 - val_acc: 0.5156 - val_precision: 0.9722 - val_recall: 0.1027 - val_fscore: 0.1850
Epoch 3/16
886s - loss: 0.4190 - acc: 0.5752 - precision: 0.6540 - recall: 0.3293 - fscore: 0.4264 - val_loss: 0.3728 - val_acc: 0.6191 - val_precision: 0.6361 - val_recall: 0.5200 - val_fscore: 0.5720
Epoch 4/16
884s - loss: 0.4248 - acc: 0.5708 - precision: 0.6988 - recall: 0.2842 - fscore: 0.3898 - val_loss: 0.4586 - val_acc: 0.5391 - val_precision: 0.6726 - val_recall: 0.0449 - val_fscore: 0.0825
Epoch 5/16


/home/cathyw/miniconda3/lib/python3.5/site-packages/keras/engine/training.py:1480: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


921s - loss: 0.4241 - acc: 0.5720 - precision: 0.7711 - recall: 0.1851 - fscore: 0.2916 - val_loss: 0.4127 - val_acc: 0.5861 - val_precision: 0.8428 - val_recall: 0.2137 - val_fscore: 0.3390
Epoch 6/16
885s - loss: 0.3992 - acc: 0.5952 - precision: 0.6671 - recall: 0.4182 - fscore: 0.5077 - val_loss: 0.3294 - val_acc: 0.6621 - val_precision: 0.6459 - val_recall: 0.5641 - val_fscore: 0.5975
Epoch 7/16
884s - loss: 0.4108 - acc: 0.5815 - precision: 0.5724 - recall: 0.6474 - fscore: 0.6022 - val_loss: 0.3705 - val_acc: 0.6230 - val_precision: 0.6929 - val_recall: 0.4294 - val_fscore: 0.5270
Epoch 8/16
884s - loss: 0.3867 - acc: 0.6074 - precision: 0.6300 - recall: 0.4976 - fscore: 0.5502 - val_loss: 0.3294 - val_acc: 0.6641 - val_precision: 0.6649 - val_recall: 0.7002 - val_fscore: 0.6807
Epoch 9/16
890s - loss: 0.3933 - acc: 0.6021 - precision: 0.6190 - recall: 0.6295 - fscore: 0.6182 - val_loss: 0.3445 - val_acc: 0.6442 - val_precision: 0.6006 - val_recall: 0.8306 - val_fscore: 0.6952
E